# Trying to use json format

some useful ressources :
  - https://blog.datafiniti.co/4-reasons-you-should-use-json-instead-of-csv-2cac362f1943
  - http://sametmax.com/yaml-xml-json-csv-ini-quest-ce-que-cest-et-a-quoi-ca-sert/
  - https://docs.python.org/2/library/json.html?highlight=json#json
  - https://www.dataquest.io/blog/python-json-tutorial/
  
 - conda install -c conda-forge ijson
 - conda install -c ioos folium

In [7]:
import sys
sys.path.insert(0,"/home/albert7a/lib/python")


## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
import time
from dask.diagnostics import ProgressBar
from datetime import date
import yaml
import io
import json
import ijson


In [148]:
diren4="/media/extra/DATA/ARGO/"
list_filesEN4=['EN.4.2.0.f.profiles.g10.201207.nc']

for f in np.arange(len(list_filesEN4)):
    fileEN4=list_filesEN4[f]
    tfileEN4=diren4+fileEN4

    dsen4=xr.open_dataset(tfileEN4)
    laten4=dsen4['LATITUDE']
    lonen4=dsen4['LONGITUDE']
    dayen4=dsen4['JULD']
    refen4=dsen4['DC_REFERENCE']
    
    print len(laten4.values)

    for ref in np.arange(len(laten4.values)):
            if ref == 0:
                dictyml={str(refen4[ref].values):{'file':fileEN4,'profile no':int(ref),'latitude':float(laten4[ref].values),'longitude':float(lonen4[ref].values),'date':str(dayen4[ref].values)}}
            else:
                up={str(refen4[ref].values):{'file':fileEN4,'profile no':int(ref),'latitude':float(laten4[ref].values),'longitude':float(lonen4[ref].values),'date':str(dayen4[ref].values)}}
                dictyml.update(up)

                
with io.open('data.yaml', 'w'i ) as outfile:
    %time yaml.dump(dictyml, outfile, default_flow_style=False, allow_unicode=True)



45254
CPU times: user 20.5 s, sys: 96 ms, total: 20.6 s
Wall time: 20.6 s


In [149]:
diren4="/media/extra/DATA/ARGO/"
list_filesEN4=['EN.4.2.0.f.profiles.g10.201207.nc']

for f in np.arange(len(list_filesEN4)):
    fileEN4=list_filesEN4[f]
    tfileEN4=diren4+fileEN4

    dsen4=xr.open_dataset(tfileEN4)
    laten4=dsen4['LATITUDE']
    lonen4=dsen4['LONGITUDE']
    dayen4=dsen4['JULD']
    refen4=dsen4['DC_REFERENCE']
    
    print len(laten4.values)

    for ref in np.arange(len(laten4.values)):
            if ref == 0:
                dictyml={int(ref+1):{'reference':str(refen4[ref].values),'file':fileEN4,'profile no':int(ref),'latitude':float(laten4[ref].values),'longitude':float(lonen4[ref].values),'date':str(dayen4[ref].values)}}
            else:
                up={int(ref+1):{'reference':str(refen4[ref].values),'file':fileEN4,'profile no':int(ref),'latitude':float(laten4[ref].values),'longitude':float(lonen4[ref].values),'date':str(dayen4[ref].values)}}
                dictyml.update(up)
               
dataf={'profilesList':dictyml}

               
with open('data.json', 'w') as f:
     %time json.dump(dictyml, f,sort_keys=True,indent=4, separators=(',', ': '))
        
 

45254
CPU times: user 832 ms, sys: 16 ms, total: 848 ms
Wall time: 848 ms


In [145]:
sourcefile=open('data.json','rU')
%time truc=json.load(sourcefile)


CPU times: user 296 ms, sys: 4 ms, total: 300 ms
Wall time: 299 ms


In [144]:
%time datapd=pd.read_json('data.json', orient='index')

CPU times: user 2.52 s, sys: 76 ms, total: 2.59 s
Wall time: 2.53 s


In [150]:
sourcefile=open('data.yaml','rU')
%time truc=yaml.load(sourcefile)


CPU times: user 32.8 s, sys: 300 ms, total: 33.1 s
Wall time: 33.1 s


In [129]:
def parse_float(x):
    try:
        x = float(x)
    except Exception:
        x = 0
    return x
datapd["latitude"]=datapd["latitude"].apply(parse_float)
datapd["longitude"]=datapd["longitude"].apply(parse_float)

In [153]:
import folium
from folium import plugins

stops_heatmap = folium.Map(location=[45,-50], zoom_start=5)
stops_heatmap.add_children(plugins.HeatMap([[row["latitude"], row["longitude"]] for name, row in datapd.iterrows()]))
stops_heatmap.save("heatmap_profilesEN4_201207.html")


In [152]:
stops_map = folium.Map(location=[45,-50], zoom_start=5)
marker_cluster = folium.MarkerCluster().add_to(stops_map)
for name, row in datapd.iterrows():
    folium.Marker([row["latitude"], row["longitude"]], popup=row["reference"]).add_to(marker_cluster)
stops_map.save("number_profilesEN4_201207.html")
